# Scraping GAMEREACTOR

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from newspaper import Source
from newspaper import fulltext

## Scraping Articles Site 

In [2]:
url = 'https://www.gamereactor.es/analisis/'
html = requests.get(url).content
html[:500]

b' <!DOCTYPE html>\n<html lang="es">\n<head>\n\n\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n\tnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n\tj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n\t\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n\t})(window,document,\'script\',\'dataLayer\',\'GTM-P9ZBFRD\');</script>\n\n\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0,'

In [3]:
soup = BeautifulSoup(html, 'lxml')
articles = soup.find('section', {'id':'textlist'}).find_all('article')

In [9]:
#Extraer títulos

titles = [article.find('h3').text for article in articles]
    
titles[0:3]

['Super Mario Bros. 35 - Battle Royale',
 "Crash Bandicoot 4: It's About Time",
 'Star Wars: Squadrons']

In [8]:
#Extraer links

links = [f"https://www.gamereactor.es{article.find_all('a')[1]['href']}" for article in articles]

print(links[0:5])


['https://www.gamereactor.es/super-mario-bros-35-battle-royale/?sid=1aded7c96af22385dd9c8d18e1388a85', 'https://www.gamereactor.es/crash-bandicoot-4-its-about-time-analisis/?sid=1aded7c96af22385dd9c8d18e1388a85', 'https://www.gamereactor.es/star-wars-squadrons-analisis/?sid=1aded7c96af22385dd9c8d18e1388a85', 'https://www.gamereactor.es/art-of-rally-analisis/?sid=1aded7c96af22385dd9c8d18e1388a85', 'https://www.gamereactor.es/kirby-fighters-2-analisis/?sid=1aded7c96af22385dd9c8d18e1388a85']


In [7]:
#Not totally useful, discarded

authors = []
for article in articles:
    full_authors = article.find_all('a', {'class': 'username'})
    for author in full_authors:
        authors.append(author.text)
    
authors[0:3]

['Sergio Figueroa', 'Eirik Hyldbakk Furu', 'Mike Holmes']

## Page parsing and link retrieving function for Gamereactor

In [10]:
def gamereactor_link_retrieve(num_pages):
    links = []
    
    # pages parser
    for i in range(num_pages):
        url = f'https://www.gamereactor.es/analisis/?page={i+1}'

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find('section', {'id':'textlist'}).find_all('article')
    

        # links retrieve
        for article in articles:
            links.append(f"https://www.gamereactor.es{article.find_all('a')[1]['href']}")
        
    #print(links)           
    return links


In [13]:
num_pages = 2

links = gamereactor_link_retrieve(num_pages)

print(links[45:55])

['https://www.gamereactor.es/peaky-blinders-mastermind-la-precuela/?sid=41207817edbc809d2c015b9bb6369924', 'https://www.gamereactor.es/mortal-shell-un-souls-endurecido/?sid=41207817edbc809d2c015b9bb6369924', 'https://www.gamereactor.es/hyper-scape-analisis/?sid=41207817edbc809d2c015b9bb6369924', 'https://www.gamereactor.es/microsoft-flight-simulator-vuelo-de-altura/?sid=41207817edbc809d2c015b9bb6369924', 'https://www.gamereactor.es/total-war-saga-troy-analisis/?sid=41207817edbc809d2c015b9bb6369924', 'https://www.gamereactor.es/hearthstone-academia-scholomance/?sid=67de3796f24dd764b65b1d94b3e6c008', 'https://www.gamereactor.es/fast-furious-crossroads-analisis/?sid=67de3796f24dd764b65b1d94b3e6c008', 'https://www.gamereactor.es/fall-guys-ultimate-knockout-analisis/?sid=67de3796f24dd764b65b1d94b3e6c008', 'https://www.gamereactor.es/horizon-zero-dawn-complete-edition-para-pc/?sid=67de3796f24dd764b65b1d94b3e6c008', 'https://www.gamereactor.es/skully-analisis/?sid=67de3796f24dd764b65b1d94b3e6

## Scraping Single Review

In [130]:
review_url = 'https://www.gamereactor.es/crash-bandicoot-4-its-about-time-analisis/'
review_html = requests.get(review_url).content
review_html[:50]

b' <!DOCTYPE html>\n<html lang="es">\n<head>\n\n\n<script'

In [131]:
soup = BeautifulSoup(review_html, 'lxml')

In [132]:
#Text + cleaning
article = soup.find('div', {'class': 'breadtext'}).find('div')
p_tags = article.find_all('p')
review = [tag.text for tag in p_tags]
review = ' '.join(review)

In [133]:
review

'Aunque creo que Ford se pasó un poco con la caña que dio a Crash Bandicoot: Nsane Trilogy, no se puede negar que a las remasterizaciones de hace unos años les falta profundidad y les sobra imprecisión en el control y repetición de eventos para estar a altura de los mejores plataformas de hoy en día. Eso no fue impedimento para recibir con alegría la noticia de que habría Crash Bandicoot 4. Además, Toys for Bob prometió tomar en consideración las críticas y el feedback recibido. Tras muchas horas de juego a It\'s About Time hay que agradecer que lo hayan hecho, a pesar de que aún queden ciertos inconvenientes del pasado. Nada más arrancar quedan presenten parte de los cambios más significativos, como puede ser el nuevo estilo visual y el rollo aún más chulesco. Todos los objetos, ya aparezcan en un vídeo cinematográfico o en la propia partida, saltan a la vista con un arco iris de color y mucho gusto por el detalle. Hay una similitud muy positiva con ese formato que recuerda a los Loon

In [134]:
# Author

author = soup.find('li', {'class': 'publishAuthor bullet'}).text
                   
author

'Eirik Hyldbakk Furu'

In [135]:
# Game

News_article = Article(review_url)
News_article.download()
News_article.parse()
News_article.title


"Crash Bandicoot 4: It's About Time"

In [136]:
# Genre, Company and Platform

infobox = soup.find_all('ul', {'class': 'infobox'})[0].contents

for box in infobox:
    
    if 'Probado en:' in box.contents[0].text:
        platform = box.contents[1]
        
    if 'Género:' in box.contents[0].text:
        genre = box.find('a').text
        
    if 'Editor:' in box.contents[0].text:
        company = box.find('a').text

print(platform, genre, company)        


 PS4, Xbox One Plataformas Activision


In [137]:
# Score

score = soup.find('div', {'class': 'bigScoreWrapper'}).find('img')['alt']
score = float(score)

In [138]:
score

8.0

## Create columns

In [140]:

def gamereactor_dict(links):
    reviews_dict = {}
    i = 0

    for link in links:

        #Request content and wait
        review_html = requests.get(link).content
        
        #Avoid get banned and timeout
        time.sleep(2)

        #Make a Soup and a Newspaper paper & build
        soup = BeautifulSoup(review_html, 'lxml')
        News_article = Article(link)
        News_article.download()
        News_article.parse()
        paper = newspaper.build(link)

        #Site name - From Newspaper
        site = paper.brand

        #URL - From Newspaper
        url_link = paper.url

        #Author - From Scraping
        author = author = soup.find('li', {'class': 'publishAuthor bullet'}).text

        #Game - From Scraping
        game = News_article.title
        
        #Company, Genre & Platform - From Scraping
        
        genre = 'None'
        company = 'None'
        platform = 'None'
        
        infobox = soup.find_all('ul', {'class': 'infobox'})[0].contents
        for box in infobox:

            if 'Probado en:' in box.contents[0].text:
                platform = box.contents[1]

            if 'Género:' in box.contents[0].text:
                genre = box.find('a').text

            if 'Editor:' in box.contents[0].text:
                company = box.find('a').text
                            

        #Text & Cleaning - From Scraping
        article = soup.find('div', {'class': 'breadtext'}).find('div')
        p_tags = article.find_all('p')
        review = [tag.text for tag in p_tags]
        review = ' '.join(review)

        #Score & Clean & Transform - From Scraping
        score = soup.find('div', {'class': 'bigScoreWrapper'}).find('img')['alt']
        score = float(score)

        #Add to a dict
        reviews_dict[i] = {'site': site,
                           'url_link': url_link,
                           'author': author,
                           'game': game,
                           'company': company,
                           'genre': genre,
                           'platform': platform,
                           'text': review,
                           'score': score}
        
        i = i+1

    return reviews_dict

In [141]:
result = gamereactor_dict(links[0:5])
print(result)

{0: {'site': 'gamereactor', 'url_link': 'https://www.gamereactor.es/super-mario-bros-35-battle-royale/?sid=41207817edbc809d2c015b9bb6369924', 'author': 'Sergio Figueroa', 'game': 'Super Mario Bros. 35', 'company': 'Nintendo', 'genre': 'Plataformas', 'platform': ' Nintendo Switch', 'text': 'No se habían olvidado de él, lo que pasaba es que era una fiesta sorpresa. El 35º cumpleaños de Super Mario Bros. comenzó con aquel Direct repleto de anuncios que miraban el pasado y se sigue celebrando con el paso de las semanas, lanzamiento a lanzamiento. Tras el calmado Super Mario 3D All-Stars, toca ponerse las pilas con el eléctrico Super Mario Bros. 35. Un clásico cuyo gameplay no pasa de moda es capaz de dar aún más de sí con unos retoques de lo más modernos, los del battle royale. Es el regalo de Nintendo para sus fans, o más bien para sus clientes, pues solo se puede acceder a él "gratis" si se está pagando la suscripción a Nintendo Switch Online. Nace como nació Tetris 99, del que lo hereda

In [145]:
result[3]

{'site': 'gamereactor',
 'url_link': 'https://www.gamereactor.es/art-of-rally-analisis/?sid=41207817edbc809d2c015b9bb6369924',
 'author': 'Petter Hegevall',
 'game': 'Art of Rally',
 'company': 'Funselektor Labs Inc',
 'genre': 'Carreras',
 'platform': ' PC',
 'text': 'Este estilo tan estilo con el que Art of Rally representa este deporte te hace pararte a pensar. Se conduce con el teclado, como hace 25 años, con una sola mano. Su tempo lento te va metiendo en un especie de hipnosis ambiental, como si el cerebro se enchufara directamente al PC. Qué pocos juegos de coches consiguen esta sensación tan relajante, aunque en el fondo se trate de ir todo lo rápido posible con un coche de rallies grupo B de los años 80 sobre la engañosa gravilla, el peligroso hielo o el asfalto. Y sí es como Funselektor Labs, que ya tienen en su haber Absolute Drift, han conseguido este feel tan único y diferenciado, que difícilmente se puede equiparar a cualquier otra cosa del presente. Art of Rally trata de

In [33]:
gamereactor = pd.DataFrame({'title': titles, 
                           'author': np.nan, 
                           'review': reviews_col, 
                           'url': links 
                           })

gamereactor.head()

,title,author,review,url
0,Análisis Bounty Battle; el título de lucha de ...,NaN,Después del anuncio de su retraso indefinido e...,https://as.com/meristation/2020/09/22/analisis...
1,"eFootball PES 2021, análisis. Rueda el balón, ...",NaN,La temporada futbolística acabó de manera atíp...,https://as.com/meristation/2020/09/22/analisis...
2,"Milky Way Prince: The Vampire Prince, análisis...",NaN,El amor es uno de los temas universales que pe...,https://as.com/meristation/2020/09/21/analisis...
3,"Spinch, análisis PC y Switch: Una experiencia ...",NaN,No hay duda de que el género de las plataforma...,https://as.com/meristation/2020/09/20/analisis...
4,"Spelunky 2, análisis: el videojuego más hardco...",NaN,Mucho ha llovido desde el lanzamiento de Spelu...,https://as.com/meristation/2020/09/19/analisis...


In [34]:
#meristation.to_csv('../data/meristation_data.csv', index=False)